In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Put postgreSQL credeitials into sql_login.py
from sql_login import USERNAME as USERNAME
from sql_login import PASSWORD as PASSWORD

import matplotlib.pyplot as plt

In [2]:
lime_df = pd.read_csv('Resources/lime.csv')
lime_df

,Unnamed: 0,Partner,Unnamed: 2,HS Code,Product,Year,UOM,Q1(Jan-Mar) Value,Q1(Jan-Mar) Qty,Q2(Apr-Jun) Value,Q2(Apr-Jun) Qty,Q3(Jul-Sep) Value,Q3(Jul-Sep)Qty,Q4(Oct-Dec) Value,Q4(Oct-Dec) Qty,Total Value,Total Qty,Reporter Code,Partner Code,Product Code
0,1,Mexico,1,805503000,"LIM,TAH/PR,FR/DR",2015-2015,MT,"82,086","101,270.30","70,518","118,700.50","65,554","139,964.90","52,877","117,020.30","271,035","476,955.90",US,MX,805503000
1,1,Mexico,1,805503000,"LIM,TAH/PR,FR/DR",2016-2016,MT,"70,928","113,589.30","126,874","128,207.70","81,396","146,864.20","69,181","130,708.00","348,379","519,369.10",US,MX,805503000
2,1,Mexico,1,805503000,"LIM,TAH/PR,FR/DR",2017-2017,MT,"111,125","116,851.50","85,032","156,135.40","87,353","162,139.10","84,084","134,692.20","367,594","569,818.20",US,MX,805503000
3,1,Mexico,1,805503000,"LIM,TAH/PR,FR/DR",2018-2018,MT,"140,423","117,758.60","127,912","151,146.10","93,635","161,657.10","73,466","145,703.10","435,436","576,265.00",US,MX,805503000
4,1,Mexico,1,805503000,"LIM,TAH/PR,FR/DR",2019-2019,MT,"113,771","137,236.80","119,491","160,433.30","119,449","164,323.60","100,280","145,816.10","452,990","607,809.90",US,MX,805503000


In [14]:
# This function outputs a data frame from a csv file from https://apps.fas.usda.gov/
# It expects data from four quarters from 2015 to 2020
def fas_csv_to_pd_df(file_name):
    input_df = pd.read_csv(file_name)
    out_df = pd.DataFrame(columns = ['Year','Quarter','Value','Qty'])
    year_list = []
    quarter_list=[]
    for j in range(2015,2020):
        for i in range(1,5):
            quarter_list.append(i)
            year_list.append(j)

    out_df['Year'] = year_list
    out_df['Quarter'] = quarter_list

    value_list = []
    quantity_list = []
    for i in range(0,5):
        for j in range(7,15,2):   
            value_list.append(input_df.iloc[i,j].replace(',',''))
            quantity_list.append(input_df.iloc[i,j+1].replace(',',''))
    out_df['Value'] = value_list
    out_df['Qty'] = quantity_list
    out_df.columns = map(str.lower, out_df.columns)
    return out_df

In [17]:
lime_df = fas_csv_to_pd_df('Resources/lime.csv')
lime_df[['value','qty']] = lime_df[['value','qty']].apply(pd.to_numeric)
lime_df['price'] = lime_df['value']/lime_df['qty']
lime_df

,year,quarter,value,qty,price
0,2015,1,82086,101270.3,0.810563
1,2015,2,70518,118700.5,0.594083
2,2015,3,65554,139964.9,0.468360
3,2015,4,52877,117020.3,0.451862
4,2016,1,70928,113589.3,0.624425
5,2016,2,126874,128207.7,0.989597
6,2016,3,81396,146864.2,0.554226
7,2016,4,69181,130708.0,0.529279
8,2017,1,111125,116851.5,0.950993
9,2017,2,85032,156135.4,0.544604


In [18]:
rds_connection_string = USERNAME+':'+PASSWORD+"@localhost:5432/avocados"
engine = create_engine(f'postgresql://{rds_connection_string}')
conn = engine.engine.connect()

In [19]:
# Drop avocado price table if it exists
conn.execute('DROP TABLE IF EXISTS lime_fsda;')

In [23]:
# Create table to hold daily avocado price data
conn.execute(
"""
    CREATE TABLE lime_fsda (
    year INT,
    quarter INT,
    qty DECIMAL(20,2),
    price double precision,
    primary key (quarter, year)
    );
"""
)

In [24]:
lime_df[['year','quarter','qty','price']].to_sql(name='lime_fsda', con=engine, if_exists='append', index=False)